In [2]:
!pip install -qU "semantic-router[fastembed]" langchain langchain_community==0.2.6 fastembed==0.3.2 langchain_core openai pymilvus bs4 "grpcio<=1.63.0,>=1.49.1" 

In [3]:
import requests
from bs4 import BeautifulSoup
import os
import time
from langchain_core.documents import Document
from urllib.parse import urlparse, urljoin
from openai import OpenAI
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection, connections, utility
from typing import Any, List, Tuple, Dict, Literal, Optional
from pydantic import Field
from semantic_router.schema import DocumentSplit
from langchain_core.documents import Document
from semantic_router.splitters import RollingWindowSplitter
from semantic_router.utils.logger import logger
from semantic_router.encoders import FastEmbedEncoder
from semantic_router.encoders import OpenAIEncoder
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_76/2354882201.py", line 8, in <module>
    from pymilvus import FieldSchema, CollectionSchema, DataType, Collection, connections, utility
  File "/opt/conda/lib/python3.10/site-packages/pymilvus/__init__.py", line 13, in <module>
    from .client import __version__
  File "/opt/conda/lib/python3.10/site-packages/pymilvus/client/__init__.py", line 15, in <module>
    __version__ = get_distribution("pymilvus").version
  File "/opt/conda/lib/python3.10/site-packages/pkg_resources/__init__.py", line 478, in get_distribution
    # through to the default implementation
  File "/opt/conda/lib/python3.10/site-packages/pkg_resources/__init__.py", line 354, in get_provider
  File "/opt/conda/lib/python3.10/site-packages/pkg_resources/__init__.py", line 909, in require
    processed

In [4]:
def get_main_links(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')
  main_links = []

  # Extracting links with "index.html" considering nesting
  for a in soup.find_all('a', href=True):
    href = a['href']
    if href and href.endswith("index.html"):  # Check for ending with "index.html"
      full_url = urljoin(url, href)
      # Avoid duplicate links and links pointing to external domains
      if full_url not in main_links and full_url.startswith(url):
        main_links.append(full_url)

  return main_links

In [12]:
response = requests.get("https://www.ey.com/en_in/supply-chain")
soup = BeautifulSoup(response.content, 'html.parser')

In [20]:
target_div = soup.find('div', class_='contentGridGeneral component section default-style col-xs-12')

# Extract all links (anchor tags) within the target div
if target_div:
    links = target_div.find_all('a', href=True)
    for link in links:
        print(link['href'])
else:
    print("Target div not found!")

In [22]:
from bs4 import BeautifulSoup
import requests

# URL of the page
url = 'https://www.ey.com/en_in/supply-chain'  # Replace with the actual URL

# Send a GET request to the webpage
response = requests.get(url)
response.raise_for_status()  # Check if the request was successful

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Locate the element
content_grid_element = soup.find('div', class_='content-grid animation-part-on container grid is-hidden')

# Get the content
content = content_grid_element.decode_contents()  # Or use .text for just the text content

print(content)


<h3 class="content-grid-title">Our latest thinking</h3>
<div class="row"></div>
<div class="controls">
<div class="is-hidden buttons">
<button class="show-more">Show more</button>
</div>
<div class="is-hidden pagination">
<button class="is-disabled prev"><span class="arrow"></span>Previous</button>
<div class="pagination-list"></div>
<button class="is-disabled next"><span class="arrow"></span>Next</button>
</div>
</div>



In [23]:
import requests
from bs4 import BeautifulSoup

# URL of the blog
url = "https://www.ey.com/en_in/technology/sustainable-coding-is-the-need-for-a-greener-tomorrow"

# Send a request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract the author name
    author_div = soup.find('div', class_='surfaceProfile-author-description')
    author_name = author_div.get_text(strip=True) if author_div else 'Author not found'
    
    # Extract related topics
    related_topics_div = soup.find('div', class_='col-xs-12 col-sm-8 col-sm-offset-1 col-md-12 col-md-offset-0 col-lg-offset-0 default-style')
    related_topics = related_topics_div.get_text(strip=True) if related_topics_div else 'Related topics not found'
    
    # Extract page content
    content_div = soup.find('div', class_='richText component section richText-copy-block col-xs-12')
    page_content = content_div.get_text(strip=True) if content_div else 'Page content not found'
    
    # Print extracted information
    print("Author Name:", author_name)
    print("Related Topics:", related_topics)
    print("Page Content:", page_content)
else:
    print("Failed to retrieve the webpage. Status code:", response.status_code)

Author Name: Alexy ThomasByAlexy ThomasPartner, Technology Consulting, EY IndiaTechnology enthusiast, Data-driven.
Related Topics: Related topicsTechnologySustainabilityDigitalTechnology leader's agendaEmerging technology
Page Content: Software industry must adopt green coding and efficient algorithms to curb rising carbon emissions.


In [25]:
import requests
from bs4 import BeautifulSoup

# URL of the blog
url = "https://www.ey.com/en_in/technology/sustainable-coding-is-the-need-for-a-greener-tomorrow"

# Send a request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract the author name
    author_div = soup.find('div', class_='surfaceProfile-author-description')
    author_name = author_div.get_text(strip=True) if author_div else 'Author not found'
    
    # Extract related topics
    related_topics_div = soup.find('div', class_='col-xs-12 col-sm-8 col-sm-offset-1 col-md-12 col-md-offset-0 col-lg-offset-0 default-style')
    related_topics = related_topics_div.get_text(strip=True) if related_topics_div else 'Related topics not found'
    
    # Extract page content
    def extract_markdown_content(div):
        markdown_content = []
        for element in div:
            if element.name == 'h1':
                markdown_content.append(f"# {element.get_text(strip=True)}")
            elif element.name == 'h2':
                markdown_content.append(f"## {element.get_text(strip=True)}")
            elif element.name == 'p':
                markdown_content.append(f"{element.get_text(strip=True)}")
            elif element.name == 'ul':
                for li in element.find_all('li', recursive=False):
                    markdown_content.append(f"- {li.get_text(strip=True)}")
            elif element.name == 'li':
                markdown_content.append(f"- {element.get_text(strip=True)}")
        return "\n".join(markdown_content)
    
    # Navigate through the hierarchy to find content
    main_div = soup.find('div', class_='optional-components paragraphSystem')
    if main_div:
        content_divs = main_div.find_all('div', class_='richText component section richText-copy-block col-xs-12')
        markdown_content = []
        for content_div in content_divs:
            inner_div = content_div.find('div', class_='component-content')
            if inner_div:
                rich_text_div = inner_div.find('div', class_='richText-content')
                if rich_text_div:
                    markdown_content.append(extract_markdown_content(rich_text_div.children))
        
        page_content = "\n\n".join(markdown_content)
    else:
        page_content = 'Page content not found'
    
    # Extract PDF download links
    pdf_links = []
    pdf_divs = soup.find_all('div', class_='fileList-download')
    for pdf_div in pdf_divs:
        pdf_link_tag = pdf_div.find('a', class_='fileList-download-link')
        if pdf_link_tag:
            pdf_link = pdf_link_tag.get('href')
            if pdf_link:
                pdf_links.append(pdf_link)
    
    # Print extracted information
    print("Author Name:", author_name)
    print("Related Topics:", related_topics)
    print("Page Content (Markdown):\n", page_content)
    print("PDF Links:", pdf_links)
else:
    print("Failed to retrieve the webpage. Status code:", response.status_code)

Author Name: Alexy ThomasByAlexy ThomasPartner, Technology Consulting, EY IndiaTechnology enthusiast, Data-driven.
Related Topics: Related topicsTechnologySustainabilityDigitalTechnology leader's agendaEmerging technology
Page Content (Markdown):
 

In May 2021, industry giants Microsoft, Thoughtworks, Accenture, and GitHub teamed up with the Joint Development Foundation Projects and The Linux Foundation to launch the Green Software Foundation. This non-profit is laser-focused on building a community for eco-friendly software development. The driving forces? A growing corporate awareness of the energy toll exacted by software development and operation— a pressing concern in our digital space. Until recently, sustainability in software and architectures took a back seat, with many companies mistakenly assuming that, unlike hardware, software did not pose environmental challenges. However, this perception shifted as it became clear that while software does not directly consume energy, po